In [93]:
import pandas as pd

In [94]:
import seaborn as sns
import matplotlib.pyplot as plt

In [95]:
df = pd.read_parquet('./data/green_tripdata_2021-01.parquet')

In [96]:
df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)

In [97]:
df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds()/60)

In [102]:
#df = df[df.trip_type == 2]

In [103]:
df.duration.describe()

count    640.000000
mean      15.112292
std       11.514934
min        1.016667
25%        6.433333
50%       12.766667
75%       21.070833
max       58.966667
Name: duration, dtype: float64

In [104]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

In [106]:
categorical = ['PULocationID','DOLocationID']
numerical = ['trip_distance']

In [111]:
df[categorical] = df[categorical].astype(str)

In [112]:
df.dtypes

VendorID                          int64
lpep_pickup_datetime     datetime64[us]
lpep_dropoff_datetime    datetime64[us]
store_and_fwd_flag               object
RatecodeID                      float64
PULocationID                     object
DOLocationID                     object
passenger_count                 float64
trip_distance                   float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
ehail_fee                        object
improvement_surcharge           float64
total_amount                    float64
payment_type                    float64
trip_type                       float64
congestion_surcharge            float64
duration                        float64
dtype: object